In [150]:
import sbmlcore, pandas
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
a = sbmlcore.SNAP2('tests/3pl1-snap2.csv')

offsets = None, are you sure?


In [95]:
b = pandas.read_csv('tests/3pl1-snap2.csv')
b

,Variant,Predicted Effect,Score,Expected Accuracy
0,M1A,effect,39,66%
1,M1R,effect,44,71%
2,M1N,effect,59,75%
3,M1D,effect,74,85%
4,M1C,effect,9,53%
...,...,...,...,...
3715,S186S,neutral,-79,87%
3716,S186T,neutral,-73,87%
3717,S186W,effect,45,71%
3718,S186Y,effect,38,66%


In [ ]:
#Thoughts - filter for above 50% accuracy? 
#When loading multiple chains - use csv_segid_concat.ipynb to concatenate into one .csv with the correct segid for each chain

In [8]:
c = sbmlcore.SNAP2('tests/5uh6-complete.csv', offsets = {'A': 0, 'B': 0, 'C': -6, 'D':0, 'E':0, 'F':0})

In [10]:
#Remove entries with less than 50% accuracy
snap2_df
#c['Expected Accuracy']

NameError: name 'snap2_df' is not defined

In [96]:
c = pandas.read_csv('tests/5uh6-complete.csv')
offsets = {'A': 0, 'B': 0, 'C': -6, 'D':0, 'E':0, 'F':0}
c

,Variant,Predicted Effect,Score,Expected Accuracy,segid
0,M1A,effect,56,75%,A
1,M1R,effect,77,85%,A
2,M1N,effect,69,80%,A
3,M1D,effect,80,91%,A
4,M1C,effect,37,66%,A
...,...,...,...,...,...
76515,D528S,effect,32,66%,F
76516,D528T,effect,33,66%,F
76517,D528W,effect,62,80%,F
76518,D528Y,effect,55,75%,F


In [97]:
#Remove entries with less than 80% accuracy
c = pandas.read_csv('tests/5uh6-complete.csv')

#First need to change 'Expected Accuracy' from strings to floats
no_percentage = c['Expected Accuracy'].replace(to_replace='%', value='', regex=True)



#no_percentage = c['Expected Accuracy'].str.split(pat="%", expand=True)
#no_percentage.drop['1']

series = pandas.to_numeric(no_percentage) #turns str into int64 so that inequalities can be used

no_percentage[ series < 80 ] #creates series with only those rows for which accuracy < 80%
no_percentage[series < 80].index #extracts index for each of the rows for which accuracy < 80%
c.drop(no_percentage[series < 80].index, inplace=True) #removes the entries for which the indices are specified above
c
#high_acc = c.drop(no_percentage[series > 80].index, inplace=True)
#print(high_acc)

,Variant,Predicted Effect,Score,Expected Accuracy,segid
1,M1R,effect,77,85%,A
2,M1N,effect,69,80%,A
3,M1D,effect,80,91%,A
6,M1E,effect,76,85%,A
7,M1G,effect,75,85%,A
...,...,...,...,...,...
76501,D528R,effect,60,80%,F
76503,D528D,neutral,-83,93%,F
76511,D528K,effect,63,80%,F
76514,D528P,effect,62,80%,F


In [111]:
#Apply offsets to resid 


offsets = {'A': 0, 'B': 0, 'C': -6, 'D':0, 'E':0, 'F':0}
#assert that all chains from csv must be accounted for?

def split_mutation_resid(row):
    m=row.Variant
    return(int(m[1:-1]))

c['resid'] = c.apply(split_mutation_resid, axis=1)
c['id'] = c['segid'] + c['resid'].astype(str)
c.set_index('id', inplace=True)
c

#Adds column for offsets
c["chain_offsets"] = [offsets[chain] for chain in c.segid]
c

#Applies offsets - adds them, as is also the case for Structural Features
c["mutation_resid"] = c["resid"] + c["chain_offsets"]
c

,Variant,Predicted Effect,Score,Expected Accuracy,segid,resid,chain_offsets,mutation_resid
id,,,,,,,,
A1,M1R,effect,77,85%,A,1,0,1
A1,M1N,effect,69,80%,A,1,0,1
A1,M1D,effect,80,91%,A,1,0,1
A1,M1E,effect,76,85%,A,1,0,1
A1,M1G,effect,75,85%,A,1,0,1
...,...,...,...,...,...,...,...,...
F528,D528R,effect,60,80%,F,528,0,528
F528,D528D,neutral,-83,93%,F,528,0,528
F528,D528K,effect,63,80%,F,528,0,528


In [119]:
#Add column for mutated_to_resname - i.e. the residue change as a result of the mutation
def split_mutation_toresname(row):
    return pandas.Series([row.Variant[-1]])

c[['mutated_to_resname']] = c.apply(split_mutation_toresname, axis=1)
c

#def split_mutation_toresname(row):
#            return pandas.Series([row.Variant[-1], int(row.mutation[1:-1])])

#        snap2_df[["mutated_to_resname", "resid"]] = snap2_df.apply(split_mutation_toresname, axis=1)

,Variant,Predicted Effect,Score,Expected Accuracy,segid,resid,chain_offsets,mutation_resid,mutated_to_resname,resid_again
id,,,,,,,,,,
A1,M1R,effect,77,85%,A,1,0,1,R,1
A1,M1N,effect,69,80%,A,1,0,1,N,1
A1,M1D,effect,80,91%,A,1,0,1,D,1
A1,M1E,effect,76,85%,A,1,0,1,E,1
A1,M1G,effect,75,85%,A,1,0,1,G,1
...,...,...,...,...,...,...,...,...,...,...
F528,D528R,effect,60,80%,F,528,0,528,R,528
F528,D528D,neutral,-83,93%,F,528,0,528,D,528
F528,D528K,effect,63,80%,F,528,0,528,K,528


In [171]:
df_rnap = sbmlcore.SNAP2("tests/5uh6-complete.csv", offsets = {'A': 0, 'B': 0, 'C': -6, 'D':0, 'E':0, 'F':0})

In [170]:
#Align SNAP2 scores with mutation dataset
rnap_mutations = {'segid': ['A', 'A', 'A', 'B', 'C', 'C'], 'mutation': ['I3D','S4K', 'Q5V', 'R6D', 'S450F', 'D435F']}
rnap_mutations_df = pandas.DataFrame(rnap_mutations)
rnap_mutations_df

,segid,mutation
0,A,I3D
1,A,S4K
2,A,Q5V
3,B,R6D
4,C,S450F
5,C,D435F


In [169]:
new = df_rnap.add_feature(rnap_mutations_df)
new

                                        mutation Variant Predicted Effect  \
segid mutation_resid mutated_to_resname                                     
A     3              D                       I3D     I3D          neutral   
      4              K                       S4K     S4K          neutral   
      5              V                       Q5V     NaN              NaN   
B     6              D                       R6D     R6D           effect   
C     450            F                     S450F   S456F           effect   
      435            F                     D435F   D441F           effect   

                                         Score Expected Accuracy  resid  \
segid mutation_resid mutated_to_resname                                   
A     3              D                   -73.0               87%    3.0   
      4              K                   -84.0               93%    4.0   
      5              V                     NaN               NaN    NaN   
B     6 

In [ ]:
#Might need to drop 'Expected Accuracy' and 'Predicted Effect' from df for 